In [10]:
import numpy as np
import json
from datetime import datetime

from shapely import geometry as sgeom
import geopandas as gpd
import ulmo
import pandas as pd
from collections import OrderedDict

In [36]:
#paths 
########## USER ###########
# Select modeling domain
domain = 'WY'

# SM location 
SMpath = '/nfs/attic/dfh/Aragon2/mar2020_snowmodel-dfhillWY_time/'

metFname = 'mm_WY_2018-2019.dat'

In [37]:
parFile = SMpath+'snowmodel.par'
incFile = SMpath+'code/snowmodel.inc'
codepath = SMpath+'code'
calpath = '/nfs/attic/dfh/Aragon2/Notebooks/calibration_python'
assimPath = '/nfs/attic/dfh/Aragon2/CSOassim/'+domain+'/'

#Edit the par file to set parameters with new values
def edit_par(par_dict,parameter,new_value):
    lines = open(parFile, 'r').readlines()
    if par_dict[parameter][2] == 14 or par_dict[parameter][2] == 17 \
    or par_dict[parameter][2] == 18 or par_dict[parameter][2] == 19 \
    or par_dict[parameter][2] == 93 or par_dict[parameter][2] == 95 \
    or par_dict[parameter][2] == 97 or par_dict[parameter][2] == 100 \
    or par_dict[parameter][2] == 102 or par_dict[parameter][2] == 104 \
    or par_dict[parameter][2] == 107 or par_dict[parameter][2] == 108:
        text = str(new_value)+'\n'
    else:
        text = str(new_value)+'\t\t\t!'+par_dict[parameter][1]
    lines[par_dict[parameter][2]] = text
    out = open(parFile, 'w')
    out.writelines(lines)
    out.close()
    
#function to edit SnowModel Files other than .par
def replace_line(file_name, line_num, text):
    lines = open(file_name, 'r').readlines()
    lines[line_num] = text
    out = open(file_name, 'w')
    out.writelines(lines)
    out.close()
    
#path to CSO domain json
with open('/nfs/attic/dfh/Aragon2/CSOdata/CSO_domains.json') as f:
    domains = json.load(f)
    
#base .par
with open('/nfs/attic/dfh/Aragon2/CSOdmn/par_base.json') as f:
    base = json.load(f)

# SM run assim CSO 

In [48]:
CSO_obs = assimPath+'CSO_assim_2018-2019_all_minelevbias10.csv'
CSO_gdf = pd.read_csv(CSO_obs)
CSO_gdf

,Unnamed: 0,Unnamed: 0.1,geometry,id,author,depth,source,timestamp,elevation,x_idx,...,M,D,LON,LAT,H,swe,doy,cf,SMswe,bias
0,0,135,POINT (-110.597818510855 43.6701724330154),D3gyiOmE,Katie O’Connell,10.000000,MountainHub,2019-04-25T02:25:30.037Z,2131.137695,452,...,4,25,-110.597819,43.670172,100.000000,0.033626,207,0.601282,0.000000,0.033626
1,21,156,POINT (-110.595210157438 43.6710762419213),aYRjNEsq,Katie O’Connell,80.000000,MountainHub,2019-04-16T16:54:40.684Z,2140.001465,454,...,4,16,-110.595210,43.671076,800.000000,0.269209,198,0.808998,0.168529,0.100680
2,22,157,POINT (-110.605929521947 43.6673063133319),M4jq7Ceg,Leanne,67.000000,MountainHub,2019-04-16T16:53:30.360Z,2112.297607,445,...,4,16,-110.605929,43.667306,670.000000,0.224520,198,0.807870,0.147879,0.076641
3,23,158,POINT (-110.6062185 43.6674586),aGXBcdMr,Megan Hedley,64.999998,MountainHub,2019-04-16T16:50:12.369Z,2111.702637,445,...,4,16,-110.606219,43.667459,649.999976,0.217782,198,0.805939,0.150554,0.067228
4,24,159,POINT (-110.589866841624 43.6727580275064),4mkXg3k0,Katie O’Connell,55.000000,MountainHub,2019-04-16T16:44:00.688Z,2144.277832,458,...,4,16,-110.589867,43.672758,550.000000,0.186311,198,0.810025,0.229347,-0.043036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,243,378,POINT (-110.770101919873 43.6852315999974),p0PtXgEu,James Molloy,66.040000,MountainHub,2018-12-17T16:55:42.771Z,2267.802490,313,...,12,17,-110.770102,43.685232,660.400000,0.166510,78,1.001184,0.169596,-0.003085
62,266,401,POINT (-110.956834076491 43.4899387327936),JoqoGJ0g,michael mourar,83.000000,MountainHub,2018-12-10T01:14:32.348Z,2656.023682,162,...,12,10,-110.956834,43.489939,830.000000,0.206241,71,0.866944,0.187805,0.018436
63,281,416,POINT (-110.870306 43.590676),8qzaI9zc,Cody McInnes,140.000000,SnowPilot,2018-12-05T19:40:00.000Z,2918.032715,232,...,12,5,-110.870306,43.590676,1400.000000,0.344194,66,0.258239,0.253802,0.090392
64,303,438,POINT (-110.010724 43.733182),53nEfN14,Chris McCollister,46.000000,SnowPilot,2018-11-14T23:00:00.000Z,2847.268799,924,...,11,14,-110.010724,43.733182,460.000000,0.096710,45,1.000000,0.064113,0.032597


In [49]:
#1 year simulations 
outFpath = assimPath+'fixxx.dat'
f= open(outFpath,"w+")

tot_obs=len(CSO_gdf)
uq_day = np.unique(CSO_gdf.dt)
num_days = len(uq_day)
ids = 100
f.write('{:02.0f}\n'.format(num_days))
for j in range(len(uq_day)):
    obs = CSO_gdf[CSO_gdf['dt']==uq_day[j]]
    d=CSO_gdf.D[CSO_gdf['dt']==uq_day[j]].values
    m=CSO_gdf.M[CSO_gdf['dt']==uq_day[j]].values
    y=CSO_gdf.Y[CSO_gdf['dt']==uq_day[j]].values
    date = str(y[0])+' '+str(m[0])+' '+str(d[0])
    obs_count = str(len(obs))
    f.write(date+' \n')
    f.write(obs_count+' \n')
    for k in range(len(obs)):
        ids = ids+1
        x= obs.x[obs.index[k]]
        y=obs.y[obs.index[k]]
        swe=obs.swe[obs.index[k]]
        f.write('{:3.0f}\t'.format(ids)+'{:10.0f}\t'.format(x)+'{:10.0f}\t'.format(y)+'{:3.2f}\n'.format(swe))
f.close() 

In [45]:
CSO_obs

'fixed.csv'

In [46]:
CSOdata = assimPath+'CSO_assim_2018-2019_all_minelevbias.dat'

assimpath = SMpath+'swe_assim/swe_obs_test.dat'
! cp $CSOdata $assimpath

In [47]:
replace_line(incFile, 30, '      parameter (max_obs_dates='+str(len(CSO_gdf)+1)+')\n')
edit_par(base,'print_var_18','y')#swed
edit_par(base,'print_var_19','y')#sspr
edit_par(base,'print_var_20','y')#ssmt
edit_par(base,'irun_data_assim',1)

%cd $codepath
! ./compile_snowmodel.script
%cd  /nfs/attic/dfh/Aragon2/Notebooks/assimilation_python

/nfs/attic/dfh/Aragon2/mar2020_snowmodel-dfhillWY_time/code
/nfs/attic/dfh/Aragon2/Notebooks/assimilation_python
